In [1]:

#START FROM HERE For TRAINING PURPOSES START FROM THIS CELL, PROVIDED THE PICKLE FILE WAS PREVIOUSLY CREATED

 
import tensorflow as tf
import numpy as np
import random
import time  
import matplotlib.pyplot as plt
from PIL import Image
import os
import sys
import scipy.io as scp
import sklearn as skl
import  urllib.request as urlreq
import random
from IPython.display import display, Image
import tarfile
from datetime import datetime
from six.moves import cPickle as pickle
%matplotlib inline
print ("Tensor flow version is -- ", tf.__version__)
tf.test.is_gpu_available()

Tensor flow version is --  1.0.0


True

In [2]:

pickle_file =  'multi_image_cnn.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels  = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels  = save['valid_labels']
    test_dataset  = save['test_dataset']
    test_labels   = save['test_labels']
    del save  

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (230000, 32, 32, 1) (230000, 6)
Validation set (5755, 32, 32, 1) (5755, 6)
Test set (13068, 32, 32, 1) (13068, 6)


In [3]:
def new_weights(shape, name):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1), name = name)
def new_biases(length , name):
    return tf.Variable(tf.constant(1.0, shape=[length]), name = name)
 
#  images are 32 pixels in each dimension.
image_size = 32 
num_channels = 1 
num_classes = 11  
batch_size= 25 

#convolution layer1
filter_size = 5 #5x5 pixels
num_filters1 = 16 #and 16 filters

#convolution layer2
num_filters2 = 32

#convolution layer3
num_filters3 = 64

#convolution layer4
num_filters4 = 128

fc_size1 = 256 
fc_size2 = 64 

graph = tf.Graph()

with graph.as_default():

    # Input parameters
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels), name="batch_data")
    tf_train_labels = tf.placeholder(tf.int64, shape=(batch_size, 6), name="batch_label")
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights1 = new_weights(shape=[filter_size, filter_size, num_channels, num_filters1] , name='weights1')  
    biases1  = new_biases (length=num_filters1, name='biases1')  


    weights2 = new_weights(shape=[filter_size, filter_size, num_filters1, num_filters2],name= 'weights2') 
    biases2  = new_biases (length=num_filters2, name='biases2')  


    weights3 = new_weights(shape=[filter_size, filter_size, num_filters2, num_filters3] , name='weights3') 
    biases3  = new_biases (length=num_filters3, name='biases3')  
    
    weights4 = new_weights(shape=[filter_size, filter_size, num_filters3, num_filters4] , name='weights3') 
    biases4  = new_biases (length=num_filters4, name='biases3')  

    weights_fc1 = new_weights(shape=[ 2*2*num_filters4, fc_size1], name ='weights_fc1') 
    biases_fc1 = new_biases (length=fc_size1, name='biases_fc1')  
    
    weights_fc2 = new_weights(shape=[ fc_size1, fc_size2], name ='weights_fc2') 
    biases_fc2 = new_biases (length=fc_size2, name='biases_fc2')  
    
    weights_d0  = new_weights(shape=[  fc_size2, num_classes],name='weights_d0') 
    biases_d0   = new_biases (length=num_classes,name='biases_d0')  
    weights_d1  = new_weights(shape=[  fc_size2, num_classes],name='weights_d1') 
    biases_d1   = new_biases (length=num_classes,name='biases_d1')  
    weights_d2  = new_weights(shape=[  fc_size2, num_classes],name='weights_d2') 
    biases_d2   = new_biases (length=num_classes,name='biases_d2')  
    weights_d3  = new_weights(shape=[  fc_size2, num_classes],name='weights_d3') 
    biases_d3   = new_biases (length=num_classes,name='biases_d3')  
    weights_d4  = new_weights(shape=[  fc_size2, num_classes],name='weights_d4') 
    biases_d4   = new_biases (length=num_classes,name='biases_d4')  
    weights_d5  = new_weights(shape=[  fc_size2, num_classes],name='weights_d5') 
    biases_d5   = new_biases (length=num_classes,name='biases_d5')  
 
    tf.summary.histogram('weights1',weights1) 
    tf.summary.histogram('weights_d0',weights_d0) 
    
    with tf.name_scope('conv_model'):
        def conv_model(data, keep_prob=1):
        
            print ( "Input Data shape ", tf.Print(data,[data]) )   

            with tf.name_scope('layer1'):
        # LAYER 1  converts 32 x 32 x 1  to  28 x 28 x 16        
                layer   = tf.nn.conv2d(input=data,  filter=weights1, strides=[1, 1, 1, 1],  padding='VALID') 
                layer0  = tf.nn.relu(layer+biases1)
                lrn     = tf.nn.local_response_normalization(layer0) 
                print ( "Conv. Layer 1 shape ", lrn.shape)   



        # LAYER 2 converts 28 x 28 x 16  to 24 x 24 x 32  and then max pools 
            with tf.name_scope('layer2'):
                layer   = tf.nn.conv2d(input=lrn,  filter=weights2, strides=[1, 1, 1, 1],  padding='VALID') 
                layer0  = tf.nn.relu(layer+biases2)
                lrn     = tf.nn.local_response_normalization(layer0) 
                print ( "Conv. Layer 2.0 shape ", lrn.shape)   
                maxpool = tf.nn.max_pool(value=lrn, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
                print ( "Conv. Layer 2.1 shape ", maxpool.shape)   


        # LAYER 3 converts 12 x 12 x 32  to 8 x 8 x 64                 
            with tf.name_scope('layer3'):
                layer   = tf.nn.conv2d(input=maxpool,  filter=weights3, strides=[1, 1, 1, 1],  padding='VALID') 
                layer0  = tf.nn.relu(layer+biases3) 
                lrn     = tf.nn.local_response_normalization(layer0) 
                print ( "Conv. Layer 3 shape ", lrn.shape)        
          
        # LAYER 4 converts 8 x 8 x 64  to 4 x 4x 128 and then max pool     
            with tf.name_scope('layer4'):
                layer   = tf.nn.conv2d(input=lrn,  filter=weights4, strides=[1, 1, 1, 1],  padding='VALID') 
                layer0  = tf.nn.relu(layer+biases4)
                lrn     = tf.nn.local_response_normalization(layer0) 
                print ( "Conv. Layer 4.0 shape ", lrn.shape)  
                maxpool = tf.nn.max_pool(value=lrn, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
                print ( "Conv. Layer 4.1 shape ", maxpool.shape)   
                           
        # Flatten the previous layer  2 x 2 x 128  to 1 x 512
                dropout        =   tf.nn.dropout(maxpool, keep_prob=keep_prob)     
                dropout_shape  =   dropout.get_shape().as_list()
                num_features   =   dropout_shape[1] * dropout_shape[2] * dropout_shape[3]
                layer_flat     =   tf.reshape(dropout, [dropout_shape[0], num_features]) 
                print ( "After Flattening Layer 4 shape ", layer_flat.shape," and num of features ", num_features)


        #   Fully connected Layer 1  with RELU 
            with tf.name_scope('FC_Layer1'):     
                layer_fc1      =  tf.nn.relu(tf.matmul(layer_flat, weights_fc1) + biases_fc1 )
                print ( "Fully connected Layer 1 shape ", layer_fc1.shape)       


        #   Fully connected Layer 2  with RELU 
            with tf.name_scope('FC_Layer2'):     
                layer_fc2      =  tf.nn.relu(tf.matmul(layer_fc1, weights_fc2) + biases_fc2 )
                print ( "Fully connected Layer 2 shape ", layer_fc2.shape)       


        # LAYER FINAL  Fully connected 

            with tf.name_scope('Final_FC_Layer'):
                length = tf.matmul(layer_fc2, weights_d0) + biases_d0
                digit1 = tf.matmul(layer_fc2, weights_d1) + biases_d1
                digit2 = tf.matmul(layer_fc2, weights_d2) + biases_d2
                digit3 = tf.matmul(layer_fc2, weights_d3) + biases_d3
                digit4 = tf.matmul(layer_fc2, weights_d4) + biases_d4
                digit5 = tf.matmul(layer_fc2, weights_d5) + biases_d5
                print ( "FINAL Fully connected Layer 3 shape - length ", length.shape)  
                print ( "FINAL Fully connected Layer 3 shape - digit1 ", digit1.shape)  
                print ( "FINAL Fully connected Layer 3 shape - digit2 ", digit2.shape)  
                print ( "FINAL Fully connected Layer 3 shape - digit3 ", digit3.shape)  
                print ( "FINAL Fully connected Layer 3 shape - digit4 ", digit4.shape)  
                print ( "FINAL Fully connected Layer 3 shape - digit5 ", digit5.shape)  
                print (  "\n\n" ) 

            return [length, digit1, digit2, digit3, digit4, digit5]    
     
    
    [length, digit1, digit2, digit3, digit4, digit5] = conv_model(tf_train_dataset, 0.9)
    
    
    with tf.name_scope ('loss'):
        loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=length, labels=tf_train_labels[:,0])) +\
                    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=digit1, labels=tf_train_labels[:,1])) +\
                    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=digit2, labels=tf_train_labels[:,2])) +\
                    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=digit3, labels=tf_train_labels[:,3])) +\
                    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=digit4, labels=tf_train_labels[:,4])) +\
                    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=digit5, labels=tf_train_labels[:,5]))
    
    # Define optimizaer and learning rate
        global_step = tf.Variable(0)
        learning_rate = 0.01
        optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss, global_step=global_step)
        tf.summary.scalar("loss",loss)
    
    # Predictions for the training, validation, and test data.    
    
    
    with tf.name_scope ('valid_accuracy'):
        valid_prediction = tf.stack([tf.nn.softmax(conv_model(tf_valid_dataset, 1.0)[0]),\
                      tf.nn.softmax(conv_model(tf_valid_dataset, 1.0)[1]),\
                      tf.nn.softmax(conv_model(tf_valid_dataset, 1.0)[2]),\
                      tf.nn.softmax(conv_model(tf_valid_dataset, 1.0)[3]),\
                      tf.nn.softmax(conv_model(tf_valid_dataset, 1.0)[4]),\
                      tf.nn.softmax(conv_model(tf_valid_dataset, 1.0)[5])])  
        temp1 = tf.transpose (tf.argmax(valid_prediction,axis=2) ) 
        temp2 = tf.equal( temp1 ,  valid_labels) 
        temp3 = tf.reduce_min ( tf.to_float(temp2 ), axis=1 )
        valid_acc_num = 100.0*tf.reduce_mean(temp3)
        tf.summary.scalar('valid_accuracy_num',valid_acc_num)
        
        
        temp4 = tf.reduce_sum ( tf.to_float(temp2 ) )
        valid_acc_dig  = 100.0*temp4/tf.to_float( ( valid_labels.shape[0]* valid_labels.shape[1] ) ) 
        tf.summary.scalar('valid_accuracy_dig',valid_acc_dig)

     
    test_prediction = tf.stack([tf.nn.softmax(conv_model(tf_test_dataset, 1.0)[0]),\
             tf.nn.softmax(conv_model(tf_test_dataset, 1.0)[1]),\
             tf.nn.softmax(conv_model(tf_test_dataset, 1.0)[2]),\
             tf.nn.softmax(conv_model(tf_test_dataset, 1.0)[3]),\
             tf.nn.softmax(conv_model(tf_test_dataset, 1.0)[4]),\
             tf.nn.softmax(conv_model(tf_test_dataset, 1.0)[5])])
    temp1 = tf.transpose (tf.argmax(test_prediction,axis=2) ) 
    temp2 = tf.equal( temp1 , test_labels) 

    temp3 = tf.reduce_min ( tf.to_float(temp2 ), axis=1 )
    test_acc_num = 100.0*tf.reduce_mean(temp3)

    temp4 = tf.reduce_sum ( tf.to_float(temp2 ) )
    test_acc_dig  = 100.0*temp4/tf.to_float( (test_labels.shape[0]*test_labels.shape[1] ) )  
        
    
    with tf.name_scope ('training_accuracy'):
        train_prediction = tf.stack([tf.nn.softmax(conv_model(tf_train_dataset, 1.0)[0]),\
                  tf.nn.softmax(conv_model(tf_train_dataset, 1.0)[1]),\
                  tf.nn.softmax(conv_model(tf_train_dataset, 1.0)[2]),\
                  tf.nn.softmax(conv_model(tf_train_dataset, 1.0)[3]),\
                  tf.nn.softmax(conv_model(tf_train_dataset, 1.0)[4]),\
                  tf.nn.softmax(conv_model(tf_train_dataset, 1.0)[5])])    
        temp1 = tf.transpose (tf.argmax(train_prediction,axis=2) ) 
        temp2 = tf.equal( temp1 , tf_train_labels) 

        temp3 = tf.reduce_min ( tf.to_float(temp2 ), axis=1 )
        train_acc_num = 100.0*tf.reduce_mean(temp3)
        tf.summary.scalar('training_accuracy_num',train_acc_num)

        temp4 = tf.reduce_sum ( tf.to_float(temp2 ) )
        train_acc_dig  = 100.0*temp4/tf.to_float( (tf_train_labels.shape[0]*tf_train_labels.shape[1] ) ) 
        tf.summary.scalar('training_accuracy_dig',train_acc_dig)
        
     
    # Merge all summaries into a single operator
    merged  = tf.summary.merge_all()
       
    #Save model
    saver = tf.train.Saver()
    
    

Input Data shape  Tensor("Print:0", shape=(25, 32, 32, 1), dtype=float32)
Conv. Layer 1 shape  (25, 28, 28, 16)
Conv. Layer 2.0 shape  (25, 24, 24, 32)
Conv. Layer 2.1 shape  (25, 12, 12, 32)
Conv. Layer 3 shape  (25, 8, 8, 64)
Conv. Layer 4.0 shape  (25, 4, 4, 128)
Conv. Layer 4.1 shape  (25, 2, 2, 128)
After Flattening Layer 4 shape  (25, 512)  and num of features  512
Fully connected Layer 1 shape  (25, 256)
Fully connected Layer 2 shape  (25, 64)
FINAL Fully connected Layer 3 shape - length  (25, 11)
FINAL Fully connected Layer 3 shape - digit1  (25, 11)
FINAL Fully connected Layer 3 shape - digit2  (25, 11)
FINAL Fully connected Layer 3 shape - digit3  (25, 11)
FINAL Fully connected Layer 3 shape - digit4  (25, 11)
FINAL Fully connected Layer 3 shape - digit5  (25, 11)



Input Data shape  Tensor("valid_accuracy/Print:0", shape=(5755, 32, 32, 1), dtype=float32)
Conv. Layer 1 shape  (5755, 28, 28, 16)
Conv. Layer 2.0 shape  (5755, 24, 24, 32)
Conv. Layer 2.1 shape  (5755, 12, 12, 3

In [5]:

# starting session and run computation 
 
num_steps = 300001
print('Training start time: ', str(datetime.now()))
with tf.Session(graph=graph,config=tf.ConfigProto(log_device_placement=True)) as session:
    
 
    file_writer = tf.summary.FileWriter('./board_multi_deep', session.graph)  
    tf.global_variables_initializer().run()
    
    
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size),:]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels } 
        _, l, tr_acc_num,tr_acc_dig = session.run([optimizer, loss,train_acc_num, train_acc_dig], feed_dict=feed_dict)
        if (step % 500 == 0 or ( step <500  and step %100 == 0) ): 
            print('Step No.' , step, '  and loss: ',l)
            print( 'Training batch accuracy for number:     %3.2f' % (tr_acc_num )  )
            print( 'Training batch accuracy for digits:     %3.2f' % (tr_acc_dig   ) )
            print( 'Validation batch accuracy for number:   %3.2f' % (valid_acc_num.eval()   ))
            print( 'Validation batch accuracy for digits:   %3.2f' % (valid_acc_dig.eval()   ))
            print( ' \n') 
             
            _, merged_summ = session.run([optimizer,merged], feed_dict=feed_dict)
            file_writer.add_summary ( merged_summ,step )
    print( 'TEST batch accuracy for number:   %3.2f' % (test_acc_num.eval()   ))
    print( 'TEST batch accuracy for digits:   %3.2f' % (test_acc_dig.eval()   ))
    save_path = saver.save(session, "SVHN_multi_deep.ckpt")
    print('Training end time: ', str(datetime.now()))
    print("Model saved in file: %s" % save_path)

Training start time:  2017-03-20 16:56:08.347071
Initialized
Step No. 0   and loss:  18.0379
Training batch accuracy for number:     0.00
Training batch accuracy for digits:     1.33
Validation batch accuracy for number:   0.00
Validation batch accuracy for digits:   45.71
 

Step No. 100   and loss:  7.91988
Training batch accuracy for number:     0.00
Training batch accuracy for digits:     56.00
Validation batch accuracy for number:   0.00
Validation batch accuracy for digits:   54.30
 

Step No. 200   and loss:  7.90509
Training batch accuracy for number:     0.00
Training batch accuracy for digits:     58.67
Validation batch accuracy for number:   0.00
Validation batch accuracy for digits:   54.30
 

Step No. 300   and loss:  7.57401
Training batch accuracy for number:     0.00
Training batch accuracy for digits:     52.67
Validation batch accuracy for number:   0.00
Validation batch accuracy for digits:   54.05
 

Step No. 400   and loss:  8.30071
Training batch accuracy for numb